In [2]:
import mediapipe as mp
import cv2
import tensorflow
print("MediaPipe version:", mp.__version__)
print("OpenCV version:", cv2.__version__)
print("Tensorflow version:", tensorflow.__version__)




MediaPipe version: 0.10.21
OpenCV version: 4.11.0
Tensorflow version: 2.15.0


In [6]:
import os
import csv
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import ffmpeg
from tqdm import tqdm # For visualizing progress
from sklearn.preprocessing import MinMaxScaler
import time
import sys

# --- 1. CONFIGURATION ---
# Define the root directory for your raw data
data_root_dir = "data"

# Define the desired standard video format
standard_fps = 30
standard_resolution = "640:480" # Width:Height

# List of folders and their corresponding CSV file names
csv_files_info = {
    "training": "how2sign_realigned_train.csv",
    "testing": "how2sign_realigned_test.csv",
    "validation": "how2sign_realigned_val.csv"
}

# The subfolder containing the raw videos
raw_videos_subfolder = "raw_videos"

# Output directory for the final processed NumPy arrays and CSVs
OUTPUT_DATA_DIR = 'processed_gru_data'
os.makedirs(OUTPUT_DATA_DIR, exist_ok=True)

# Define Hyperparameters for the GRU Model Input
SEQUENCE_LENGTH = 60

# Initialize MediaPipe Holistic for extracting all landmarks (hands, face, pose)
mp_holistic = mp.solutions.holistic

# --- 2. HELPER FUNCTIONS ---

def extract_landmarks(results):
    """
    Extracts and flattens all available MediaPipe landmarks into a single 1D array.
    """
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() \
        if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z, res.visibility] for res in results.face_landmarks.landmark]).flatten() \
        if results.face_landmarks else np.zeros(468*4)
    lh = np.array([[res.x, res.y, res.z, res.visibility] for res in results.left_hand_landmarks.landmark]).flatten() \
        if results.left_hand_landmarks else np.zeros(21*4)
    rh = np.array([[res.x, res.y, res.z, res.visibility] for res in results.right_hand_landmarks.landmark]).flatten() \
        if results.right_hand_landmarks else np.zeros(21*4)
    return np.concatenate([pose, face, lh, rh])

def process_and_extract_sequence(video_path, holistic_model):
    """
    Reads a video and extracts the sequence of landmark features.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None
    frame_sequence = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic_model.process(image_rgb)
        keypoints = extract_landmarks(results)
        frame_sequence.append(keypoints)
    cap.release()
    return np.array(frame_sequence)

def normalize_and_pad_sequence(sequence, scaler, max_length=SEQUENCE_LENGTH):
    """Normalizes coordinates and pads/truncates the sequence to a fixed length."""
    if sequence.size == 0:
        return np.zeros((max_length, scaler.n_features_in_))
    normalized_sequence = scaler.transform(sequence)
    if normalized_sequence.shape[0] < max_length:
        padding_needed = max_length - normalized_sequence.shape[0]
        padding = np.zeros((padding_needed, normalized_sequence.shape[1]))
        padded_sequence = np.concatenate([normalized_sequence, padding], axis=0)
    elif normalized_sequence.shape[0] > max_length:
        padded_sequence = normalized_sequence[:max_length, :]
    else:
        padded_sequence = normalized_sequence
    return padded_sequence
    
# --- 3. MAIN EXECUTION ---
def main():
    
    # --- Phase 1: Preprocessing and Aligning Raw Data ---
    print("--- Phase 1: Preprocessing and Aligning Raw Data ---")
    
    all_video_data_from_processed_dirs = []
    
    for folder_name, csv_file_name in csv_files_info.items():
        input_videos_dir = os.path.join(data_root_dir, folder_name, raw_videos_subfolder)
        input_csv_path = os.path.join(data_root_dir, folder_name, csv_file_name)
        
        processed_videos_dir = os.path.join(data_root_dir, f"processed_{folder_name}", "videos")
        processed_csv_dir = os.path.join(data_root_dir, f"processed_{folder_name}")
        os.makedirs(processed_videos_dir, exist_ok=True)
        os.makedirs(processed_csv_dir, exist_ok=True)

        print(f"Aligning and processing '{folder_name}'...")
        
        if not os.path.exists(input_videos_dir) or not os.path.exists(input_csv_path):
            print(f"Warning: Raw data not found for '{folder_name}'. Skipping.")
            continue

        df_raw = pd.read_csv(input_csv_path, sep='\t')
        
        processed_rows = []
        video_id_counter = 0

        raw_video_files = [f for f in os.listdir(input_videos_dir) if f.endswith((".mp4", ".mov", ".avi", ".mkv"))]
        raw_video_names_without_ext = {os.path.splitext(f)[0] for f in raw_video_files}
        
        df_aligned = df_raw[df_raw['SENTENCE_NAME'].isin(raw_video_names_without_ext)].copy()

        for index, row in tqdm(df_aligned.iterrows(), total=len(df_aligned), desc=f"Standardizing {folder_name} videos"):
            video_name = row['SENTENCE_NAME']
            label = row['SENTENCE']
            
            video_file_with_ext = next((f for f in raw_video_files if os.path.splitext(f)[0] == video_name), None)
            if video_file_with_ext is None:
                continue
                
            input_video_path = os.path.join(input_videos_dir, video_file_with_ext)
            output_video_path = os.path.join(processed_videos_dir, f"standardized_{video_name}.mp4")

            try:
                (
                    ffmpeg
                    .input(input_video_path)
                    .output(output_video_path, vf=f"fps={standard_fps},scale={standard_resolution}", vcodec="libx264", crf=20)
                    .run(overwrite_output=True, quiet=True)
                )
                
                processed_row = {
                    'video_id': video_id_counter,
                    'SENTENCE_NAME': f"standardized_{video_name}",
                    'SENTENCE': label
                }
                processed_rows.append(processed_row)
                video_id_counter += 1

            except ffmpeg.Error:
                print(f"Warning: FFmpeg standardization failed for {video_name}. Skipping.")
                continue

        processed_df = pd.DataFrame(processed_rows)
        processed_df.to_csv(os.path.join(processed_csv_dir, f"processed_{folder_name}.csv"), sep='\t', index=False)
        print(f"Saved processed CSV for {folder_name} with {len(processed_rows)} aligned rows.")
        
        for row in processed_rows:
            video_path = os.path.join(processed_videos_dir, row['SENTENCE_NAME'] + '.mp4')
            all_video_data_from_processed_dirs.append((video_path, row['SENTENCE'], folder_name))

    print("\nAll preprocessing and alignment complete.")
    print(f"Total videos found after preprocessing: {len(all_video_data_from_processed_dirs)}")
    
    # --- Phase 2: First Pass - Extract & fit scaler on training data ---
    print("\n--- Phase 2: Fitting Normalizer on training data ---")
    training_data_raw = []
    training_data = [item for item in all_video_data_from_processed_dirs if item[2] == 'training']

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for video_path, _, _ in tqdm(training_data, desc="Extracting Raw Training Keypoints"):
            raw_sequence = process_and_extract_sequence(video_path, holistic)
            if raw_sequence is not None and raw_sequence.size > 0:
                training_data_raw.append(raw_sequence)

    if not training_data_raw:
        print("No training data found. Cannot fit scaler. Exiting.")
        return
    
    combined_train_frames = np.vstack(training_data_raw)
    scaler = MinMaxScaler()
    scaler.fit(combined_train_frames)
    print(f"Normalizer (MinMaxScaler) fitted on {combined_train_frames.shape[0]} training frames.")

    # --- Phase 3: Second Pass - Process, Normalize, and Save ---
    print("\n--- Phase 3: Normalizing, Padding, and Saving Sequences ---")
    final_data = {'training': {'X': [], 'y': []}, 
                  'testing': {'X': [], 'y': []}, 
                  'validation': {'X': [], 'y': []}}
    unique_labels = sorted(list(set(item[1] for item in all_video_data_from_processed_dirs)))
    label_to_int = {label: i for i, label in enumerate(unique_labels)}

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for video_path, label, folder_type in tqdm(all_video_data_from_processed_dirs, desc="Final Processing and Saving"):
            raw_sequence = process_and_extract_sequence(video_path, holistic)
            if raw_sequence is None or raw_sequence.size == 0:
                continue
            padded_sequence = normalize_and_pad_sequence(raw_sequence, scaler)
            int_label = label_to_int[label]
            final_data[folder_type]['X'].append(padded_sequence)
            final_data[folder_type]['y'].append(int_label)
    
    for set_key, data in final_data.items():
        if data['X']:
            X_data = np.array(data['X'])
            y_data = np.array(data['y'])
            np.save(os.path.join(OUTPUT_DATA_DIR, f'X_{set_key}.npy'), X_data)
            np.save(os.path.join(OUTPUT_DATA_DIR, f'y_{set_key}.npy'), y_data)
            print(f"\nSaved {set_key} data: X_shape={X_data.shape}, y_shape={y_data.shape}")

    label_df = pd.DataFrame(list(label_to_int.items()), columns=['SENTENCE', 'LABEL_INT'])
    label_df.to_csv(os.path.join(OUTPUT_DATA_DIR, 'label_map.csv'), index=False)
    print(f"\nLabel map saved with {len(unique_labels)} unique sentences.")
    
if __name__ == '__main__':
    main()

--- Phase 1: Preprocessing and Aligning Raw Data ---
Aligning and processing 'training'...


Standardizing training videos: 100%|█████████████████████████████████████████████████| 182/182 [01:12<00:00,  2.52it/s]


Saved processed CSV for training with 182 aligned rows.
Aligning and processing 'testing'...


Standardizing testing videos: 100%|████████████████████████████████████████████████████| 84/84 [00:43<00:00,  1.93it/s]


Saved processed CSV for testing with 84 aligned rows.
Aligning and processing 'validation'...


...

Saved processed CSV for validation with 84 aligned rows.

All preprocessing and alignment complete.
Total videos found after preprocessing: 350

--- Phase 2: Fitting Normalizer on training data ---


Extracting Raw Training Keypoints: 100%|███████████████████████████████████████████| 182/182 [1:50:55<00:00, 36.57s/it]


Normalizer (MinMaxScaler) fitted on 34215 training frames.

--- Phase 3: Normalizing, Padding, and Saving Sequences ---


Final Processing and Saving: 100%|█████████████████████████████████████████████████| 350/350 [3:30:23<00:00, 36.07s/it]



Saved training data: X_shape=(182, 60, 2172), y_shape=(182,)

Saved testing data: X_shape=(84, 60, 2172), y_shape=(84,)

Saved validation data: X_shape=(84, 60, 2172), y_shape=(84,)

Label map saved with 305 unique sentences.


In [7]:
import os
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm # For visualizing progress

# --- 1. CONFIGURATION ---
# Define the root directory where your processed data resides.
ROOT_DIR = 'data'
PROCESSED_FOLDERS = ['processed_training', 'processed_testing', 'processed_validation']
# Output directory for the final processed NumPy arrays
OUTPUT_DATA_DIR = 'processed_gru_data'
os.makedirs(OUTPUT_DATA_DIR, exist_ok=True)
# Define Hyperparameters for the GRU Model Input
SEQUENCE_LENGTH = 60

# Initialize MediaPipe Holistic for extracting all landmarks (hands, face, pose)
mp_holistic = mp.solutions.holistic

# --- 2. HELPER FUNCTIONS ---
def extract_landmarks(results):
    """
    Extracts and flattens all available MediaPipe landmarks into a single 1D array.
    """
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() \
        if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z, res.visibility] for res in results.face_landmarks.landmark]).flatten() \
        if results.face_landmarks else np.zeros(468*4)
    lh = np.array([[res.x, res.y, res.z, res.visibility] for res in results.left_hand_landmarks.landmark]).flatten() \
        if results.left_hand_landmarks else np.zeros(21*4)
    rh = np.array([[res.x, res.y, res.z, res.visibility] for res in results.right_hand_landmarks.landmark]).flatten() \
        if results.right_hand_landmarks else np.zeros(21*4)
    return np.concatenate([pose, face, lh, rh])

def process_and_extract_sequence(video_path, holistic_model):
    """Reads a video and extracts the sequence of landmark features."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None
    frame_sequence = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic_model.process(image_rgb)
        keypoints = extract_landmarks(results)
        frame_sequence.append(keypoints)
    cap.release()
    return np.array(frame_sequence)

def normalize_and_pad_sequence(sequence, scaler, max_length=SEQUENCE_LENGTH):
    """Normalizes coordinates and pads/truncates the sequence to a fixed length."""
    if sequence.size == 0:
        return np.zeros((max_length, scaler.n_features_in_))
    normalized_sequence = scaler.transform(sequence)
    if normalized_sequence.shape[0] < max_length:
        padding_needed = max_length - normalized_sequence.shape[0]
        padding = np.zeros((padding_needed, normalized_sequence.shape[1]))
        padded_sequence = np.concatenate([normalized_sequence, padding], axis=0)
    elif normalized_sequence.shape[0] > max_length:
        padded_sequence = normalized_sequence[:max_length, :]
    else:
        padded_sequence = normalized_sequence
    return padded_sequence
    
# --- 3. MAIN EXECUTION ---
def main():
    all_video_data = []
    
    # --- Phase 1: Load data from already processed folders ---
    print("--- Phase 1: Loading Processed Data ---")
    for folder in PROCESSED_FOLDERS:
        # Correctly join the path for the processed CSV
        # The filename created by the first script is 'processed_training.csv', etc.
        label_csv_path = os.path.join(ROOT_DIR, folder, f"{folder}.csv") 
        videos_dir = os.path.join(ROOT_DIR, folder, "videos")
        
        if not os.path.exists(label_csv_path) or not os.path.exists(videos_dir):
            print(f"Warning: Processed data not found for '{folder}'. Skipping.")
            continue
            
        df_labels = pd.read_csv(label_csv_path, sep='\t')
        
        for index, row in df_labels.iterrows():
            video_name = row['SENTENCE_NAME']
            label = row['SENTENCE']
            video_filepath = os.path.join(videos_dir, video_name + '.mp4')
            
            if os.path.exists(video_filepath):
                all_video_data.append((video_filepath, label, folder))
            else:
                print(f"Missing video file: {video_filepath}. Skipping.")

    if not all_video_data:
        print("No video data found. Please check your paths and file names.")
        return
    print(f"Total videos found across all sets: {len(all_video_data)}")

    # --- Phase 2: First Pass - Extract & fit scaler on training data ---
    print("\n--- Phase 2: Fitting Normalizer on training data ---")
    training_data_raw = []
    training_data = [item for item in all_video_data if 'training' in item[2]]

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for video_path, _, _ in tqdm(training_data, desc="Extracting Raw Training Keypoints"):
            raw_sequence = process_and_extract_sequence(video_path, holistic)
            if raw_sequence is not None and raw_sequence.size > 0:
                training_data_raw.append(raw_sequence)

    if not training_data_raw:
        print("No training data found for scaler fitting. Please check your files.")
        return
    
    combined_train_frames = np.vstack(training_data_raw)
    scaler = MinMaxScaler()
    scaler.fit(combined_train_frames)
    print(f"Normalizer (MinMaxScaler) fitted on {combined_train_frames.shape[0]} training frames.")

    # --- Phase 3: Second Pass - Process, Normalize, and Save ---
    print("\n--- Phase 3: Normalizing, Padding, and Saving Sequences ---")
    final_data = {'training': {'X': [], 'y': []}, 
                  'testing': {'X': [], 'y': []}, 
                  'validation': {'X': [], 'y': []}}
    unique_labels = sorted(list(set(item[1] for item in all_video_data)))
    label_to_int = {label: i for i, label in enumerate(unique_labels)}

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for video_path, label, folder_type in tqdm(all_video_data, desc="Final Processing and Saving"):
            raw_sequence = process_and_extract_sequence(video_path, holistic)
            if raw_sequence is None or raw_sequence.size == 0:
                continue
            padded_sequence = normalize_and_pad_sequence(raw_sequence, scaler)
            int_label = label_to_int[label]
            key = folder_type.split('_')[1]
            final_data[key]['X'].append(padded_sequence)
            final_data[key]['y'].append(int_label)
    
    for set_key, data in final_data.items():
        if data['X']:
            X_data = np.array(data['X'])
            y_data = np.array(data['y'])
            np.save(os.path.join(OUTPUT_DATA_DIR, f'X_{set_key}.npy'), X_data)
            np.save(os.path.join(OUTPUT_DATA_DIR, f'y_{set_key}.npy'), y_data)
            print(f"\nSaved {set_key} data: X_shape={X_data.shape}, y_shape={y_data.shape}")

    label_df = pd.DataFrame(list(label_to_int.items()), columns=['SENTENCE', 'LABEL_INT'])
    label_df.to_csv(os.path.join(OUTPUT_DATA_DIR, 'label_map.csv'), index=False)
    print(f"\nLabel map saved with {len(unique_labels)} unique sentences.")
    
if __name__ == '__main__':
    main()

--- Phase 1: Loading Processed Data ---
Total videos found across all sets: 350

--- Phase 2: Fitting Normalizer on training data ---


Extracting Raw Training Keypoints: 100%|███████████████████████████████████████████| 182/182 [1:43:26<00:00, 34.10s/it]


Normalizer (MinMaxScaler) fitted on 34215 training frames.

--- Phase 3: Normalizing, Padding, and Saving Sequences ---


Final Processing and Saving: 100%|█████████████████████████████████████████████████| 350/350 [3:31:11<00:00, 36.21s/it]



Saved training data: X_shape=(182, 60, 2172), y_shape=(182,)

Saved testing data: X_shape=(84, 60, 2172), y_shape=(84,)

Saved validation data: X_shape=(84, 60, 2172), y_shape=(84,)

Label map saved with 305 unique sentences.


In [8]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

# --- 1. CONFIGURATION ---
OUTPUT_DATA_DIR = 'processed_gru_data'

# --- 2. DATA LOADING & PREPARATION ---
print("--- Loading Data ---")

X_train = np.load(os.path.join(OUTPUT_DATA_DIR, 'X_training.npy'))
y_train = np.load(os.path.join(OUTPUT_DATA_DIR, 'y_training.npy'))
X_test = np.load(os.path.join(OUTPUT_DATA_DIR, 'X_testing.npy'))
y_test = np.load(os.path.join(OUTPUT_DATA_DIR, 'y_testing.npy'))
X_val = np.load(os.path.join(OUTPUT_DATA_DIR, 'X_validation.npy'))
y_val = np.load(os.path.join(OUTPUT_DATA_DIR, 'y_validation.npy'))

label_map_df = pd.read_csv(os.path.join(OUTPUT_DATA_DIR, 'label_map.csv'))
num_classes = len(label_map_df)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"Number of classes: {num_classes}")

y_train_one_hot = to_categorical(y_train, num_classes)
y_test_one_hot = to_categorical(y_test, num_classes)
y_val_one_hot = to_categorical(y_val, num_classes)

# --- 3. BUILD AND COMPILE A SIMPLER GRU MODEL ---
print("\n--- Building and Compiling a Simpler Model ---")

num_features = X_train.shape[2]
sequence_length = X_train.shape[1]

model = Sequential()
model.add(Input(shape=(sequence_length, num_features))) 
model.add(GRU(32, return_sequences=True))  # Further reduced units
model.add(Dropout(0.7))                     # Increased dropout
model.add(GRU(16))                        # Reduced units
model.add(Dropout(0.7))                     # Increased dropout
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# --- 4. TRAIN THE MODEL ---
print("\n--- Starting Model Training ---")

# Add EarlyStopping to monitor validation loss and prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

history = model.fit(
    X_train,
    y_train_one_hot,
    epochs=100,
    batch_size=4,
    validation_data=(X_val, y_val_one_hot),
    callbacks=[early_stopping] # Add the callback here
)

print("\nModel training complete.")

# --- 5. EVALUATE THE MODEL ---
print("\n--- Final Model Evaluation on Test Data ---")
loss, accuracy = model.evaluate(X_test, y_test_one_hot, verbose=0)

print(f"\nModel Accuracy on Test Data: {accuracy * 100:.2f}%")
print(f"Model Loss on Test Data: {loss:.4f}")

--- Loading Data ---
X_train shape: (182, 60, 2172)
y_train shape: (182,)
X_test shape: (84, 60, 2172)
y_test shape: (84,)
X_val shape: (84, 60, 2172)
y_val shape: (84,)
Number of classes: 305

--- Building and Compiling a Simpler Model ---


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 60, 32)            211776    
                                                                 
 dropout (Dropout)           (None, 60, 32)            0         
                                                                 
 gru_1 (GRU)                 (None, 16)                2400      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 32)                544       
           

In [9]:
import os
import csv
import numpy as np
import pandas as pd
import cv2
import ffmpeg
from tqdm import tqdm
import time
import sys

# --- CONFIGURATION ---
data_root_dir = "data"
standard_fps = 30
standard_resolution = "640:480"
csv_files_info = {
    "training": "how2sign_realigned_train.csv",
    "testing": "how2sign_realigned_test.csv",
    "validation": "how2sign_realigned_val.csv"
}
raw_videos_subfolder = "raw_videos"

# --- HELPER FUNCTION ---
def verify_video_properties(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    return fps, (width, height)


# --- PHASE 1: PREPROCESSING & ALIGNMENT ---
print("--- Phase 1: Preprocessing and Aligning Raw Data ---")

all_video_data_from_processed_dirs = []

for folder_name, csv_file_name in csv_files_info.items():
    input_videos_dir = os.path.join(data_root_dir, folder_name, raw_videos_subfolder)
    input_csv_path = os.path.join(data_root_dir, folder_name, csv_file_name)

    processed_videos_dir = os.path.join(data_root_dir, f"processed_{folder_name}", "videos")
    processed_csv_dir = os.path.join(data_root_dir, f"processed_{folder_name}")
    os.makedirs(processed_videos_dir, exist_ok=True)
    os.makedirs(processed_csv_dir, exist_ok=True)

    print(f"Aligning and processing '{folder_name}'...")

    if not os.path.exists(input_videos_dir) or not os.path.exists(input_csv_path):
        print(f"Warning: Raw data not found for '{folder_name}'. Skipping.")
        continue

    df_raw = pd.read_csv(input_csv_path, sep='\t')

    processed_rows = []
    video_id_counter = 0

    raw_video_files = [f for f in os.listdir(input_videos_dir) if f.endswith((".mp4", ".mov", ".avi", ".mkv"))]
    raw_video_names_without_ext = {os.path.splitext(f)[0] for f in raw_video_files}

    df_aligned = df_raw[df_raw['SENTENCE_NAME'].isin(raw_video_names_without_ext)].copy()

    for index, row in tqdm(df_aligned.iterrows(), total=len(df_aligned), desc=f"Standardizing {folder_name} videos"):
        video_name = row['SENTENCE_NAME']
        label = row['SENTENCE']

        video_file_with_ext = next((f for f in raw_video_files if os.path.splitext(f)[0] == video_name), None)
        if video_file_with_ext is None:
            continue

        input_video_path = os.path.join(input_videos_dir, video_file_with_ext)
        output_video_path = os.path.join(processed_videos_dir, f"standardized_{video_name}.mp4")

        # Skip already processed
        if os.path.exists(output_video_path):
            print(f"[SKIP] {output_video_path} already exists.")
        else:
            try:
                (
                    ffmpeg
                    .input(input_video_path)
                    .output(output_video_path, vf=f"fps={standard_fps},scale={standard_resolution}",
                            vcodec="libx264", crf=20)
                    .run(overwrite_output=True, quiet=True)
                )
            except ffmpeg.Error:
                print(f"Warning: FFmpeg failed for {video_name}.")
                continue

        props = verify_video_properties(output_video_path)
        if props:
            fps, (w, h) = props
            if abs(fps - standard_fps) > 1 or (w, h) != tuple(map(int, standard_resolution.split(":"))):
                print(f"[WARNING] {video_name}: Non-standard ({fps:.1f} fps, {w}x{h})")

        processed_row = {
            'video_id': video_id_counter,
            'SENTENCE_NAME': f"standardized_{video_name}",
            'SENTENCE': label
        }
        processed_rows.append(processed_row)
        video_id_counter += 1

    processed_df = pd.DataFrame(processed_rows)
    processed_df.to_csv(os.path.join(processed_csv_dir, f"processed_{folder_name}.csv"), sep='\t', index=False)
    print(f" Saved processed CSV for {folder_name} with {len(processed_rows)} aligned rows.")

    for row in processed_rows:
        video_path = os.path.join(processed_videos_dir, row['SENTENCE_NAME'] + '.mp4')
        all_video_data_from_processed_dirs.append((video_path, row['SENTENCE'], folder_name))

print("\n All preprocessing and alignment complete.")
print(f"Total videos found after preprocessing: {len(all_video_data_from_processed_dirs)}")

# Save video list for next cell
import joblib
joblib.dump(all_video_data_from_processed_dirs, "all_video_data.pkl")
print("Saved video metadata list as 'all_video_data.pkl' for next phase.")


--- Phase 1: Preprocessing and Aligning Raw Data ---
Aligning and processing 'training'...


Standardizing training videos:   2%|█                                                  | 4/182 [00:00<00:04, 39.71it/s]

[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_0-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_1-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_10-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_11-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_12-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_13-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_14-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_15-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_16-8-rgb_front.mp4 already exists.


Standardizing training videos:   8%|███▊                                              | 14/182 [00:00<00:04, 39.81it/s]

[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_17-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_2-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_3-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_4-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_5-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_6-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_7-8-rgb_front.mp4 already exists.
[SKIP] data\processed_training\videos\standardized_-_3bUhnn4PU_8-8-rgb_front.mp4 already exists.


...

[SKIP] data\processed_validation\videos\standardized__7uBzSGPQis_4-3-rgb_front.mp4 already exists.


Standardizing validation videos:  93%|█████████████████████████████████████████████▌   | 78/84 [00:01<00:00, 39.49it/s]

[SKIP] data\processed_validation\videos\standardized__7uBzSGPQis_4-5-rgb_front.mp4 already exists.
[SKIP] data\processed_validation\videos\standardized__7uBzSGPQis_5-3-rgb_front.mp4 already exists.
[SKIP] data\processed_validation\videos\standardized__7uBzSGPQis_5-5-rgb_front.mp4 already exists.
[SKIP] data\processed_validation\videos\standardized__7uBzSGPQis_6-3-rgb_front.mp4 already exists.
[SKIP] data\processed_validation\videos\standardized__7uBzSGPQis_6-5-rgb_front.mp4 already exists.
[SKIP] data\processed_validation\videos\standardized__7uBzSGPQis_7-3-rgb_front.mp4 already exists.
[SKIP] data\processed_validation\videos\standardized__7uBzSGPQis_7-5-rgb_front.mp4 already exists.
[SKIP] data\processed_validation\videos\standardized__7uBzSGPQis_8-3-rgb_front.mp4 already exists.
[SKIP] data\processed_validation\videos\standardized__7uBzSGPQis_8-5-rgb_front.mp4 already exists.


Standardizing validation videos: 100%|█████████████████████████████████████████████████| 84/84 [00:02<00:00, 39.26it/s]

[SKIP] data\processed_validation\videos\standardized__7uBzSGPQis_9-3-rgb_front.mp4 already exists.
[SKIP] data\processed_validation\videos\standardized__7uBzSGPQis_9-5-rgb_front.mp4 already exists.
 Saved processed CSV for validation with 84 aligned rows.

 All preprocessing and alignment complete.
Total videos found after preprocessing: 350
Saved video metadata list as 'all_video_data.pkl' for next phase.


In [ ]:
import numpy as np
import cv2
import mediapipe as mp
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load
import os
import pandas as pd

# --- CONFIGURATION ---
OUTPUT_DATA_DIR = 'processed_gru_data'
SEQUENCE_LENGTH = 60
mp_holistic = mp.solutions.holistic

# --- LOAD VIDEO LIST ---
all_video_data_from_processed_dirs = load("all_video_data.pkl")

# --- HELPER FUNCTIONS ---
def extract_landmarks(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() \
        if results.pose_landmarks else np.zeros(33 * 4)
    face = np.array([[res.x, res.y, res.z, res.visibility] for res in results.face_landmarks.landmark]).flatten() \
        if results.face_landmarks else np.zeros(468 * 4)
    lh = np.array([[res.x, res.y, res.z, res.visibility] for res in results.left_hand_landmarks.landmark]).flatten() \
        if results.left_hand_landmarks else np.zeros(21 * 4)
    rh = np.array([[res.x, res.y, res.z, res.visibility] for res in results.right_hand_landmarks.landmark]).flatten() \
        if results.right_hand_landmarks else np.zeros(21 * 4)
    return np.concatenate([pose, face, lh, rh])


def process_and_extract_sequence(video_path, holistic_model):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    frame_sequence = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic_model.process(image_rgb)
        keypoints = extract_landmarks(results)
        frame_sequence.append(keypoints)
    cap.release()
    return np.array(frame_sequence) if frame_sequence else None


def normalize_and_pad_sequence(sequence, scaler, max_length=SEQUENCE_LENGTH):
    if sequence.size == 0:
        return np.zeros((max_length, scaler.n_features_in_))
    normalized_sequence = scaler.transform(sequence)
    if normalized_sequence.shape[0] < max_length:
        padding = np.zeros((max_length - normalized_sequence.shape[0], normalized_sequence.shape[1]))
        return np.concatenate([normalized_sequence, padding], axis=0)
    else:
        return normalized_sequence[:max_length, :]


# --- PHASE 2: LANDMARK EXTRACTION ---
print("\n--- Phase 2: Landmark Extraction & Normalization ---")

training_data = [v for v in all_video_data_from_processed_dirs if v[2] == 'training']
final_data = {'training': {'X': [], 'y': []},
              'testing': {'X': [], 'y': []},
              'validation': {'X': [], 'y': []}}

# Extract training data for fitting scaler
training_data_raw = []
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for video_path, _, _ in tqdm(training_data, desc="Extracting training landmarks"):
        seq = process_and_extract_sequence(video_path, holistic)
        if seq is not None and seq.size > 0:
            training_data_raw.append(seq)

if not training_data_raw:
    raise ValueError("No training sequences found. Check your video paths.")

combined_train_frames = np.vstack(training_data_raw)
scaler = MinMaxScaler().fit(combined_train_frames)
dump(scaler, os.path.join(OUTPUT_DATA_DIR, 'scaler.pkl'))
print(f" Scaler fitted and saved with {combined_train_frames.shape[0]} frames.")

# Build label map
unique_labels = sorted(set(label for _, label, _ in all_video_data_from_processed_dirs))
label_to_int = {label: i for i, label in enumerate(unique_labels)}

# --- Normalize, pad, and save all sets ---
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for video_path, label, folder_type in tqdm(all_video_data_from_processed_dirs, desc="Processing all videos"):
        seq = process_and_extract_sequence(video_path, holistic)
        if seq is None or seq.size == 0:
            continue
        seq_norm = normalize_and_pad_sequence(seq, scaler)
        final_data[folder_type]['X'].append(seq_norm)
        final_data[folder_type]['y'].append(label_to_int[label])

for set_key, data in final_data.items():
    if data['X']:
        X, y = np.array(data['X']), np.array(data['y'])
        np.save(os.path.join(OUTPUT_DATA_DIR, f'X_{set_key}.npy'), X)
        np.save(os.path.join(OUTPUT_DATA_DIR, f'y_{set_key}.npy'), y)
        print(f" Saved {set_key}: X={X.shape}, y={y.shape}")

pd.DataFrame(label_to_int.items(), columns=['SENTENCE', 'LABEL_INT']).to_csv(
    os.path.join(OUTPUT_DATA_DIR, 'label_map.csv'), index=False
)
print("\n Landmark extraction, normalization, and saving complete.")



--- Phase 2: Landmark Extraction & Normalization ---


Extracting training landmarks: 100%|███████████████████████████████████████████████| 182/182 [1:24:31<00:00, 27.86s/it]


 Scaler fitted and saved with 34215 frames.


Processing all videos:  24%|█████████████▌                                          | 85/350 [26:27<2:06:39, 28.68s/it]